In [1]:
!pip install torch torchvision

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:01<00:00, 96.1MB/s] 


In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()  # zero the parameter gradients
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 2000] loss: 2.231
[Epoch 1, Batch 4000] loss: 1.859
[Epoch 1, Batch 6000] loss: 1.693
[Epoch 1, Batch 8000] loss: 1.579
[Epoch 1, Batch 10000] loss: 1.532
[Epoch 1, Batch 12000] loss: 1.474
[Epoch 2, Batch 2000] loss: 1.421
[Epoch 2, Batch 4000] loss: 1.389
[Epoch 2, Batch 6000] loss: 1.346
[Epoch 2, Batch 8000] loss: 1.356
[Epoch 2, Batch 10000] loss: 1.333
[Epoch 2, Batch 12000] loss: 1.308
[Epoch 3, Batch 2000] loss: 1.232
[Epoch 3, Batch 4000] loss: 1.215
[Epoch 3, Batch 6000] loss: 1.240
[Epoch 3, Batch 8000] loss: 1.215
[Epoch 3, Batch 10000] loss: 1.210
[Epoch 3, Batch 12000] loss: 1.204
[Epoch 4, Batch 2000] loss: 1.099
[Epoch 4, Batch 4000] loss: 1.134
[Epoch 4, Batch 6000] loss: 1.134
[Epoch 4, Batch 8000] loss: 1.125
[Epoch 4, Batch 10000] loss: 1.156
[Epoch 4, Batch 12000] loss: 1.101
[Epoch 5, Batch 2000] loss: 1.047
[Epoch 5, Batch 4000] loss: 1.060
[Epoch 5, Batch 6000] loss: 1.047
[Epoch 5, Batch 8000] loss: 1.064
[Epoch 5, Batch 10000] loss: 1.057
[Epoc

In [5]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 61.57 %
